In [4]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

# Configure gmaps to use your Google API Key
gmaps.configure(api_key=g_key)

In [7]:
# Store the csv in a data frame
city_data_df = pd.read_csv("./weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Dmitrovsk-Orlovskiy,RU,2020-08-25 22:39:44,52.51,35.15,63.39,56,83,12.19
1,1,Ushuaia,AR,2020-08-25 22:37:33,-54.80,-68.30,42.80,56,40,6.69
2,2,Tasiilaq,GL,2020-08-25 22:39:45,65.61,-37.64,39.20,86,0,5.82
3,3,Albany,US,2020-08-25 22:39:45,42.60,-73.97,81.00,45,7,5.99
4,4,Alanya,TR,2020-08-25 22:39:45,36.54,32.00,80.46,52,0,2.37


In [8]:
# Gmaps requires data to be int or float so check dtypes
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [9]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [10]:
## Heatmap of temperature
# Get the latitude and longitude
locations = city_data_df[["Lat","Lng"]]

# Get the maximum temperature
max_temp = city_data_df["Max Temp"]

# Assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

#Assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)

#Add the heatmap layer
fig.add_layer(heat_layer)

#Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
## Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
## Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
cloud = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=cloud, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
## Heatmap of windspeed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
## Customer prompts/filtering
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [15]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Albany,US,2020-08-25 22:39:45,42.60,-73.97,81.00,45,7,5.99
4,4,Alanya,TR,2020-08-25 22:39:45,36.54,32.00,80.46,52,0,2.37
5,5,Dublin,US,2020-08-25 22:39:45,37.70,-121.94,87.01,51,0,8.01
7,7,Eyl,SO,2020-08-25 22:39:45,7.98,49.82,76.80,76,46,23.47
10,10,Dingle,PH,2020-08-25 22:39:45,11.00,122.67,78.30,84,84,3.78
13,13,Marawi,PH,2020-08-25 22:39:46,8.00,124.29,75.99,87,94,5.99
14,14,Dong Hoi,VN,2020-08-25 22:39:46,17.48,106.60,83.10,74,77,7.92
16,16,Puerto Ayora,EC,2020-08-25 22:39:46,-0.74,-90.35,77.00,53,75,26.40
21,21,Hasaki,JP,2020-08-25 22:39:47,35.73,140.83,79.00,88,20,4.70
23,23,Vaini,TO,2020-08-25 22:39:47,-21.20,-175.20,77.00,83,20,3.36


In [16]:
## Add hotel info
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Albany,US,81.00,42.60,-73.97,
4,Alanya,TR,80.46,36.54,32.00,
5,Dublin,US,87.01,37.70,-121.94,
7,Eyl,SO,76.80,7.98,49.82,
10,Dingle,PH,78.30,11.00,122.67,
13,Marawi,PH,75.99,8.00,124.29,
14,Dong Hoi,VN,83.10,17.48,106.60,
16,Puerto Ayora,EC,77.00,-0.74,-90.35,
21,Hasaki,JP,79.00,35.73,140.83,
23,Vaini,TO,77.00,-21.20,-175.20,


In [19]:
# Set parameters to search for hotel
params = {"radius":5000,
         "type": "lodging",
         "key": g_key}

# Iterate through the df
for index, row in hotel_df.iterrows():
    #Get the latitude and longitude
    lat=row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"
           
    #Use the search term lodging and our lat and long
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [21]:
##Create Pop up markers for hotel locations
# Add the pop-up box
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

#Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer =  gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# iterate through the hotel_df adding the box to the marker layer
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))